In [1]:
import pandas as pd
import os
os.chdir('/Users/soundfuze/Desktop/Jupyter/Capstone/manipulated')
data = pd.read_excel('/Users/soundfuze/Desktop/Jupyter/Capstone/Source/MSA_M2021_dl.xlsx')
data.head()

,AREA,AREA_TITLE,AREA_TYPE,PRIM_STATE,NAICS,NAICS_TITLE,I_GROUP,OWN_CODE,OCC_CODE,OCC_TITLE,...,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
0,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,00-0000,All Occupations,...,17.4,24.28,36.83,20770,25380,36180,50490,76600,NaN,NaN
1,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-0000,Management Occupations,...,37.06,51.4,74.96,37110,54620,77080,106910,155920,NaN,NaN
2,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-1021,General and Operations Managers,...,35.71,48.98,74.66,29900,45930,74270,101870,155290,NaN,NaN
3,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-2021,Marketing Managers,...,53.41,82.78,82.78,59280,77080,111080,172180,172180,NaN,NaN
4,10180,"Abilene, TX",4,TX,0,Cross-industry,cross-industry,1235,11-2022,Sales Managers,...,41.75,55.71,78.28,46650,61150,86830,115870,162820,NaN,NaN


In [5]:
# Select columns we need, and replace symbols to 0.
temp_data = pd.DataFrame(data, columns=['AREA_TITLE', 'PRIM_STATE', 'OCC_TITLE', 'H_MEAN', 'A_MEAN'])
temp_data.replace('*', 0, inplace = True)
temp_data.replace('#', 0, inplace = True)

# Calculate Annual mean wage using Mean hourly wage that columns only have Hourly mean wage.
temp_data.loc[(temp_data['H_MEAN'] != 0) & (temp_data['A_MEAN'] == 0), 'A_MEAN'] = (temp_data['H_MEAN'] * 1920).round(0)

# Drop columns neither Hourly mean wage nor Annual mean wage.
# Modify city names without State names.
temp_data = temp_data[(temp_data['H_MEAN'] != 0) & (temp_data['A_MEAN'] != 0)]
temp_data['AREA_TITLE'] = temp_data['AREA_TITLE'].apply(lambda x: x.split(',')[0])

# Rename columns.
final_data = pd.DataFrame(temp_data, columns = ['AREA_TITLE', 'PRIM_STATE', 'OCC_TITLE', 'A_MEAN'])
final_data.rename(columns = {'AREA_TITLE':'CITY', 'PRIM_STATE':'STATE(abbr)', 'OCC_TITLE':'JOB TITLE', 'A_MEAN':'Annual mean wage'}, inplace = True)
final_data

,CITY,STATE(abbr),JOB TITLE,Annual mean wage
0,Abilene,TX,All Occupations,44500
1,Abilene,TX,Management Occupations,89520
2,Abilene,TX,General and Operations Managers,84420
3,Abilene,TX,Marketing Managers,121300
4,Abilene,TX,Sales Managers,98110
...,...,...,...,...
147183,Worcester,MA,"Laborers and Freight, Stock, and Material Move...",38100
147184,Worcester,MA,Machine Feeders and Offbearers,40600
147185,Worcester,MA,"Packers and Packagers, Hand",33000
147186,Worcester,MA,Stockers and Order Fillers,35730


In [6]:
# Manipulate city names to divide the columns.
final_data['CITY'] = final_data['CITY'].apply(lambda x: x.replace("--",","))
final_data['CITY'] = final_data['CITY'].apply(lambda x: x.replace("-",","))

In [7]:
# Divide columns by city name to refine the dataset.
final_data = final_data.assign(CITY = final_data.CITY.str.split(","))
final_data = final_data.explode('CITY')
final_data.head()

,CITY,STATE(abbr),JOB TITLE,Annual mean wage
0,Abilene,TX,All Occupations,44500
1,Abilene,TX,Management Occupations,89520
2,Abilene,TX,General and Operations Managers,84420
3,Abilene,TX,Marketing Managers,121300
4,Abilene,TX,Sales Managers,98110


In [18]:
abbrevStates = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

statesList = dict(zip(abbrevStates.values(), abbrevStates.keys()))
statesList

{'AL': 'Alabama',
 'AK': 'Alaska',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DE': 'Delaware',
 'FL': 'Florida',
 'GA': 'Georgia',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PA': 'Pennsylvania',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VT': 'Vermont',
 'VA': 'Virginia',
 'WA': 'Washington',
 'WV': 'West Virginia',
 'WI': 'Wisconsin',
 'WY': 'Wyoming',
 'DC': 'District of Columbia

In [19]:
final_data['STATE'] = final_data['STATE(abbr)'].replace(statesList)
final_data.head()

,CITY,STATE(abbr),JOB TITLE,Annual mean wage,STATE
0,Abilene,TX,All Occupations,44500,Texas
1,Abilene,TX,Management Occupations,89520,Texas
2,Abilene,TX,General and Operations Managers,84420,Texas
3,Abilene,TX,Marketing Managers,121300,Texas
4,Abilene,TX,Sales Managers,98110,Texas


In [20]:
final_data = final_data[['CITY','STATE','STATE(abbr)','JOB TITLE', 'Annual mean wage']]
final_data.head()

,CITY,STATE,STATE(abbr),JOB TITLE,Annual mean wage
0,Abilene,Texas,TX,All Occupations,44500
1,Abilene,Texas,TX,Management Occupations,89520
2,Abilene,Texas,TX,General and Operations Managers,84420
3,Abilene,Texas,TX,Marketing Managers,121300
4,Abilene,Texas,TX,Sales Managers,98110


In [21]:
# Sort 'STATE', 'CITY' column by alphabetical order.
final_data = final_data.sort_values(by = ['STATE', 'CITY'])
final_data.head()

,CITY,STATE,STATE(abbr),JOB TITLE,Annual mean wage
5145,Anniston,Alabama,AL,All Occupations,42170
5146,Anniston,Alabama,AL,Management Occupations,95040
5147,Anniston,Alabama,AL,General and Operations Managers,104680
5148,Anniston,Alabama,AL,Sales Managers,105900
5149,Anniston,Alabama,AL,Computer and Information Systems Managers,118420


In [35]:
# Test codes
'''
testData1 = final_data[final_data['CITY'].str.contains('Bellevue')]
testData2 = final_data[final_data['CITY'] == 'Seattle']
testData1.head()

testData2 = testData1.assign(CITY = testData1.CITY.str.split(","))
testData2.head()

testData3 = testData2.explode('CITY')
testData3
'''

'\ntestData1 = final_data[final_data[\'CITY\'].str.contains(\'Bellevue\')]\ntestData2 = final_data[final_data[\'CITY\'] == \'Seattle\']\ntestData1.head()\n\ntestData2 = testData1.assign(CITY = testData1.CITY.str.split(","))\ntestData2.head()\n\ntestData3 = testData2.explode(\'CITY\')\ntestData3\n'

In [22]:
# Export dataset into Excel file
final_data.to_excel('SalaryData_final_1.xlsx', index = False)